In [1]:
import os
import json
from typing import Any, Optional
from llama_index.core.schema import QueryBundle
from utils import json_load, json_dump, mkdir
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama
from llama_index.core.prompts import PromptTemplate

from llama_index.core.workflow import Event
from llama_index.core.schema import TextNode
from llama_index.core.schema import NodeWithScore

from llama_index.core.workflow import (
    Workflow,
    step,
    Context,
    StartEvent,
    StopEvent,
)

from wiki_searcher import WikiSearcher
from llama_index.tools.tavily_research.base import TavilyToolSpec

# prompts

In [2]:
QUERY2KEYWORDS_PROMPT = PromptTemplate(
    template="""
你是一個專門處理中醫考題的語言模型。我將給你一道中醫考題，請從題目和選項中提取最多 5 個「專有名詞」，
這些名詞應該是適合拿去維基百科搜尋的關鍵字，也就是在維基百科上可能有條目的中醫專有名詞，例如穴位名稱、典籍名稱、病症名稱等。  

請以 JSON 格式返回，key 為 "keywords"，value 為一個字串列表。  
不要輸出其他文字或解釋。  

範例輸出：
{
  "keywords": ["四關穴", "合谷", "太衝"]
}

#{query}

請直接輸出 JSON：
"""
)

RETRIEVAL_EVALUATOR_PROMPT = PromptTemplate(
    template="""你是一個具備判斷 retrieval 資料完整性與可靠性的助手。

輸入：
- query（string）：一題單選題（例如 題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n）
- context（string）：經過 Wiki 或其它來源檢索到的 text 資料，用來支持回答這個 query

任務：
1. 判斷 context 是否足夠「正確回答」這個單選題：
   - correct：context 明確支持某個選項，沒有爭議或缺漏
   - incorrect：context 與題目方向錯誤，無法支撐任何選項
   - ambiguous：context 與題目部分相關，但缺少關鍵細節或有多種可能性尚未排除
2. 輸出 JSON 物件，格式如下（**僅包含這兩個欄位，不能有其他文字**）：

{
  "verdict": "<correct 或 incorrect 或 ambiguous>",
  "feedback": "<簡短扼要解釋你的判斷>"
}

* 如果 `verdict` 是 ambiguous，可以在 feedback 裡簡短說明缺少哪些資訊，或可提出下一步檢索方向。
* 若 `verdict` 是 incorrect，可簡短說明為何 context 不相關，必要時可提示可能的 query。

---

### 範例

query = "常見針灸配穴法中，所指的「四關穴」為何？"
context = "四關穴是指合谷穴和太衝穴的組合，常用於疏通經絡。"

輸出：

{
  "verdict": "correct",
  "feedback": "context 明確指出四關穴包含合谷與太衝，可回答問題"
}

query = "常見針灸配穴法中，所指的「四關穴」為何？"
context = "四關穴是位於手部和足部的穴位，對應心包經和肝經"

輸出：

{
  "verdict": "ambiguous",
  "feedback": "context 提到穴位位置和經絡，但未明確說明哪兩個穴位組成四關穴，建議查詢 '四關穴 穴位 組合'"
}

query = "常見針灸配穴法中，所指的「四關穴」為何？"
context = "針灸可以治療感冒和頭痛"

輸出：

{
  "verdict": "incorrect",
  "feedback": "context 與四關穴組合無關，無法支撐任何選項"
}

---

query = {query}
context = {context}"""
)

QUERY_TRANSFORM_PROMPT = PromptTemplate(
    template="""你是一個「檢索查詢重寫助手」。
任務：
根據使用者的原始題目（query）與系統回饋（feedback），產生 3 條**新的檢索查詢語句**，
讓搜尋引擎（例如 Tavily）能夠補足原本 context 缺少的資訊。

---

### 輸入
題目 (query): 原始問題

系統回饋 (feedback): 原始查詢的評估回饋

---

### 輸出要求
1. 仔細閱讀 feedback，找出原 context 缺少的資訊。
2. 根據缺口重寫或擴充查詢，使其能更有效地找到相關資料。
3. 生成 **3 條簡短、自然、可直接搜尋的查詢語句**。
4. 查詢應聚焦於 feedback 提到的重點，例如：缺少定義、組成、關聯或用法。
5. 不要重述整個題目或選項，只專注於關鍵主題。
6. 僅輸出 **有效 JSON**，格式如下：

{
  "refined_queries": [
    "查詢1",
    "查詢2",
    "查詢3"
  ]
}

若你認為只需要兩條查詢，也可以只輸出兩條。

請務必只輸出有效的 JSON，不要包含額外說明或文字。

---

### 範例輸入
query = 題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?
選項:
 A: 上星、日月
 B: 合谷、太衝
 C: 內關、外關
 D: 上關、下關

feedback = context 只描述合谷穴，未提及四關穴的組成或太衝穴，無法確定選項。
建議檢索「四關穴 組合」或查太衝穴是否與合谷一起構成四關穴。

### 預期輸出

{
  "refined_queries": [
    "四關穴 組成",
    "四關穴 包含哪些穴位",
    "合谷 太衝 四關穴 關聯"
  ]
}

---

### 輸入
題目 (query):
{query}

系統回饋 (feedback):
{feedback}

""")

DOCUMENT_EXTRACT_PROMPT = PromptTemplate(
    template="""你是精準的文本抽取器。你的任務有兩個：
1. 判定下面的 DOCUMENT 是否應保留作為 QUERY 的候選 context。
2. 如果保留，抽取最能直接回答 QUERY 的句子（1-3 句）。

條件：
- 只摘自原文（不要改寫或新增資訊）。
- 優先保留直接回答 QUERY 或含關鍵詞的句子。
- 若 DOCUMENT 與 QUERY 明顯無關、是廣告、重複或空白，回 keep=false。
- 嚴格輸出 JSON 格式，不要多餘文字。

輸出 JSON schema：
{
  "keep": true|false,                  # 是否保留 DOCUMENT
  "reason": "一句話說明為何保留或捨棄",
  "important_spans": [                  # 若保留，列出 0~3 個最關鍵短句
    "片段內容1", "片段內容2", "片段內容3"
  ]
}

輸入：
QUERY: {query}
DOCUMENT: {document}
"""
)

ANSWER_PROMPT_WITH_CONTEXT = PromptTemplate(
    template="""
你是一個中醫考題專家，請根據下面的題目回答單選題。

請遵守以下規則：
1. 嚴格依據提供的參考資料 context 作答。
2. 輸出 JSON 格式。
3. JSON 需包含兩個 key：
   - "ans" ：只回答單選答案 (A/B/C/D)
   - "feedback" ：簡短說明為什麼選這個答案
4. 不要加入題目之外的說明或其他文字。

題目：
#{query}

參考資料 (context)：
#{context}

請直接輸出 JSON：
"""
)

In [3]:
def make_context(nodes):
    context = ''
    for node in nodes:
#        context += f"{node.metadata['title']}\n"
        context += node.text
        context += '\n\n-----\n\n'
    if not nodes:
        context = '沒有找到相關結果'
    return context

In [4]:
class KeywordsEvent(Event):
    """Transformed keyword to wiki search"""

    keywords: list[str]

class RetrieveEvent(Event):
    """Retrieve event (gets retrieved nodes)."""

    retrieved_nodes: list[NodeWithScore]

class QueryEvent(Event):
    """Query event. Queries given relevant text and search text."""

    query: str
    context: str

class QueryTransformEvent(Event):
    """ """
    query: str
    feedback: str

class TavilyEvent(Event):
    """ """
    tavily_query: list[str]

class TavilyRetrieveEvent(Event):
    """ """
    retrieved_nodes: list[Any]

class ContextMergeEvent(Event):
    retrieved_nodes: list[Any]

In [5]:
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor
compressor_llmlingua2 = LongLLMLinguaPostprocessor(
    model_name="microsoft/llmlingua-2-xlm-roberta-large-meetingbank",
    device_map="auto",
    use_llmlingua2=True,
)

/home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


In [6]:
class CorrectiveRAGWorkflow(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: Optional[Any] = None,
        gemma: Optional[Any] = None,
        wiki_searcher: Optional[Any] = None,
        tavily_searcher: Optional[Any] = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.wiki_searcher = wiki_searcher
        self.tavily_searcher = tavily_searcher
        self.llm = llm
        self.gemma = gemma

    @step
    async def query2keywords(self, ctx: Context, ev: StartEvent) -> KeywordsEvent | None:
        print('query2keyword')
        query_str: str | None = ev.get("query_str")
        if query_str is None:
            return None
        await ctx.store.set('qset', query_str)
        llm = self.llm
        prompt = QUERY2KEYWORDS_PROMPT.format(query=query_str)
        response = llm.complete(prompt)
        keywords = json.loads(response.text)['keywords']
        await ctx.store.set("keywords", keywords)
        return KeywordsEvent(keywords=keywords)

    @step
    async def wiki_search(self, ctx: Context, ev: KeywordsEvent) -> RetrieveEvent | None:
        print('wiki_search')
        query = await ctx.store.get('qset')
        keywords = ev.keywords
        wiki_searcher = self.wiki_searcher
        nodes = []
        for keyword in keywords:
            rv = wiki_searcher.search_keyword(keyword)
            if rv:
                metadata = {
                    'keyword': keyword,
                    'title': rv['title'],
                    'url': rv['url'],
                    'summary': rv['summary'],
                }
                text = rv['summary']
                node = TextNode(text=text, metadata=metadata)
                score_node = NodeWithScore(node=node, score=1)
                nodes.append(score_node)
        #results = compressor_llmlingua2._postprocess_nodes(
        #    nodes, query_bundle=QueryBundle(query_str=query)
        #)
        return RetrieveEvent(retrieved_nodes=nodes)

    @step
    async def retrieval_evaluator(self, ctx: Context, ev: RetrieveEvent) -> QueryEvent | QueryTransformEvent:
        """ merge and evaluate
        """
        print('retrieval_evaluator')
        retrieved_nodes = ev.retrieved_nodes
        context = make_context(retrieved_nodes)
        query_str = await ctx.store.get("qset")
        llm = self.llm
        prompt = RETRIEVAL_EVALUATOR_PROMPT.format(query=query_str, context=context)
        response = json.loads(llm.complete(prompt).text)
        verdict = response['verdict']
        feedback = response['feedback']
        await ctx.store.set("wiki_result", retrieved_nodes)
        await ctx.store.set("evaluator_response", response)
        await ctx.store.set('wiki_context', context)
        await ctx.store.set('verdict', verdict)
        if verdict == 'correct':
            return QueryEvent(query=query_str, context=context)
        else:
            return QueryTransformEvent(query=query_str, feedback=feedback)

    @step
    async def transform_query(self, ctx: Context, ev: QueryTransformEvent) -> TavilyEvent:
        print('transform_query')
        query = ev.query
        feedback = ev.feedback
        llm = self.llm
        prompt = QUERY_TRANSFORM_PROMPT.format(query=query, feedback=feedback)
        response = json.loads(llm.complete(prompt).text)
        num_subqueries = len(response['refined_queries'])
        await ctx.store.set("num_subqueries", num_subqueries)
        return TavilyEvent(tavily_query=response['refined_queries'])

    @step 
    async def tavily_search(self, ctx: Context, ev: TavilyEvent) -> TavilyRetrieveEvent:
        print('tavily_search')
        tavily_searcher = self.tavily_searcher
        sub_querys = ev.tavily_query
        num_query = len(sub_querys)
        rvs = []
        for idx in range(num_query):
            print(f'tavily query: {idx+1}: {sub_querys[idx]}')
            search_results = tavily_searcher.search(
                sub_querys[idx], max_results=2
            )
            for search_result in search_results:
                rvs.append((sub_querys[idx], search_result))
        return TavilyRetrieveEvent(retrieved_nodes=rvs)

    @step
    async def retrieval_filter(self, ctx: Context, ev: TavilyRetrieveEvent) -> ContextMergeEvent:

        retrieved_result = ev.retrieved_nodes
        llm = self.llm
        rvs = []
        print("filtering tavily result...")
        for idx, item in enumerate(retrieved_result):
            print(f"{idx}", end=', ')
            sub_query, doc = item
            text = doc.text
            prompt = DOCUMENT_EXTRACT_PROMPT.format(query=sub_query, document=text)
            response = json.loads(llm.complete(prompt).text)
            rvs.append((response, sub_query, doc))
        return ContextMergeEvent(retrieved_nodes=rvs)

    @step
    async def merge_context(self, ctx: Context, ev: ContextMergeEvent) -> QueryEvent:
        print('merge_context')
        filter_result = ev.retrieved_nodes
        query = await ctx.store.get('qset')
        #print(f"query: {query}")  # source query
        sub_query_cache = []
        context = ''
        for fr in filter_result:
            sub_query = fr[1]
            result_dict = fr[0]
            keep = result_dict['keep']
            reason = result_dict['reason']
            spans = result_dict['important_spans']
            if keep:
                if sub_query in sub_query_cache:
                    continue
                sub_query_cache.append(sub_query)
                context += f'\n\n# 檢索：{sub_query}\n'
                context += f'# 總評：{reason}\n'
                context += "# 結果：\n"
                context += '\n- '.join(spans)

        return QueryEvent(query=query, context=context)

    @step
    async def query_result(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        print('query_result')
        """Get result with relevant text."""
        llm = self.gemma
        query = ev.query
        context = ev.context
        verdict = await ctx.store.get('verdict')

        if verdict == 'ambiguous':
            wiki_context = await ctx.store.get('wiki_context')
            context = context + '\n\n' + wiki_context

        prompt = ANSWER_PROMPT_WITH_CONTEXT.format(query=query, context=context)
        response = json.loads(llm.complete(prompt).text)
        response['context'] = context
        return StopEvent(result=response)

# visualize

In [7]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(
    CorrectiveRAGWorkflow,
    filename="custom_crag.html",
)

custom_crag.html


# data prepare

In [8]:
# get data
SOURCE_DIR = os.path.join('data', 'source')
exam_file_path = os.path.join(SOURCE_DIR, 'exam_dataset.json')

dataset = json_load(exam_file_path)['examples']
dataset = dataset[:10]

load data from: data/source/exam_dataset.json


# workflow initializer

In [9]:
gemma = Ollama(model="gemma3:12b", temperature=0.0, request_timeout=1000.0, json_mode=True)
mini = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False, response_format={"type": "json_object"})
wiki_searcher = WikiSearcher(language="zh")
tavily_searcher = TavilyToolSpec(
    api_key=TAVILY_API_KEY,
)

wf = CorrectiveRAGWorkflow(llm=mini, wiki_searcher=wiki_searcher, tavily_searcher=tavily_searcher, gemma=gemma, timeout=1000)

gio: file:///home/poyuan/workspace/rag30/days/day28/custom_crag.html: Failed to find default application for content type ‘text/html’


In [10]:
DEST_DIR = os.path.join('data', 'deliverables')
mkdir(DEST_DIR)
save_file_path = os.path.join(DEST_DIR, 'workflow_result.json')

Directory 'data/deliverables' already exists.


In [11]:
ctx = Context(wf)  # 建立空 context

In [12]:
rvs = []
for idx, data in enumerate(dataset):
    print(f"question index: {idx}")
    query = data['query']
    response = await wf.run(query_str=query)
    rv = response.copy()
    rv['query'] = query
    rv['reference_answer'] = data['reference_answer']
    rvs.append(rv)
json_dump(save_file_path, rvs)

question index: 0
query2keyword
wiki_search
retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 四關穴 組成穴位
tavily query: 2: 四關穴 是否包含 合谷 太衝
tavily query: 3: 太衝穴 位置 與合谷 配伍
filtering tavily result...
0, 1, 2, 3, 4, 5, merge_context
query_result
question index: 1
query2keyword
wiki_search
retrieval_evaluator
query_result
question index: 2
query2keyword
wiki_search
retrieval_evaluator
query_result
question index: 3
query2keyword
wiki_search
retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 外丘 國際譯名
tavily query: 2: 外丘 GB 編號
tavily query: 3: 外丘 對應 哪個 GB 穴位
filtering tavily result...
0, 1, 2, 3, 4, 5, merge_context
query_result
question index: 4
query2keyword
wiki_search
retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 陰維脈 郄穴
tavily query: 2: 陰維脈 郄穴 是 交信 還是 築賓
tavily query: 3: 交信 穴 郄穴 功能 與 陰維脈 關聯
filtering tavily result...
0, 1, 2, 3, 4, 5, merge_context
query_result
question index: 5
query2keyword
wiki_search


/home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 足太陰脾經 的絡穴是哪一穴
tavily query: 2: 公孫 SP4 是否為脾經的絡穴
tavily query: 3: 脾經 絡穴 列表
filtering tavily result...
0, 1, 2, 3, 4, 5, merge_context
query_result
question index: 6
query2keyword
wiki_search
retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 難經 第六十八難 心下滿 穴位
tavily query: 2: 難經 第六十八難 原文 註解 主心下滿
tavily query: 3: 難經 心下滿 主哪一穴
filtering tavily result...
0, 1, 2, 3, 4, 5, merge_context
query_result
question index: 7
query2keyword
wiki_search
retrieval_evaluator
query_result
question index: 8
query2keyword
wiki_search
retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 三焦經 與 膽經 交會穴 列表
tavily query: 2: 臑會 穴位 經絡 歸屬
tavily query: 3: 顴髎 秉風 聽宮 耳門 各自 屬於 哪一經絡
filtering tavily result...
0, 1, 2, 3, 4, 5, merge_context
query_result
question index: 9
query2keyword
wiki_search
retrieval_evaluator
transform_query
tavily_search
tavily query: 1: 通里 (HT5) 是否為郄穴 古代穴位分類
tavily query: 2: 通里 是否列入 馬丹陽天星十二穴
tavily quer

In [13]:
correct = 0
incorrect = 0
for rv in rvs:
    reference_answer = rv['reference_answer']
    pred_ans = rv['ans']
    if reference_answer == pred_ans:
        correct+=1
    else:
        incorrect+=1

print(f"{correct}/{correct + incorrect}")

7/10


In [15]:
rvs

[{'ans': 'B',
  'feedback': '參考資料明確指出「四關穴」由合谷與太衝兩穴組成。',
  'context': '\n\n# 檢索：四關穴 組成穴位\n# 總評：文档多处明确说明“四关穴”由合谷与太冲两对穴位组成，直接回答組成穴位的问题。\n# 結果：\n于明·徐凤所著《针灸大全》“四关者，五脏有六腑，六腑有十二原，十二原出于四关，太冲、合谷是也”。\n- 《针灸大成》中记载: “四关: 四穴，即两合谷、两太冲穴是也。”\n- 四关穴即合谷、太冲，均为原穴，开四关能开通一身气机，调节阴阳升降出入，是临床常用的两个配伍穴。\n\n# 檢索：四關穴 是否包含 合谷 太衝\n# 總評：文中多處明確記載“四關”包含合谷與太衝，可直接回答查詢。\n# 結果：\n“四关者，五脏有六腑，六腑有十二原，十二原出于四关，太冲、合谷是也”。\n- 《针灸大成》中记载: “四关: 四穴，即两合谷、两太冲穴是也。”\n- 四关穴即合谷、太冲，均为原穴，开四关能开通一身气机，调节阴阳升降出入，是临床常用的两个配伍穴。\n\n# 檢索：太衝穴 位置 與合谷 配伍\n# 總評：文件包含合谷與太冲的定位說明及二穴配伍說明，直接回答查詢內容。\n# 結果：\n合谷穴位于手背侧，第１、２掌骨间，在第２掌骨桡侧的中点处。\n- 太冲穴位于足背侧，在第１、２跖骨底结合部前方的凹陷中。\n- 四关穴即合谷、太冲，均为原穴，开四关能开通一身气机，调节阴阳升降出入，是临床常用的两个配伍穴。\n\n合谷穴（LI 4）是手陽明大腸經的原穴，出自《靈樞·本輸》，又名虎口。“合”意即合攏，“谷”是山谷的意思。此穴在第一、二掌骨之間，兩骨相合，形狀如山谷的地方，所以名為合谷。又因位於手拇指虎口兩骨之間，所以又稱為虎口。\n\n-----\n\n',
  'query': '題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n',
  'reference_answer': 'B'},
 {'ans': 'A',
  'feedback': '描述的循行路徑（從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中）與足太陽膀胱經的循行內容相符。',
  'context': '《靈樞經》，與《素